In [ ]:
import pandas as pd

df_econ = pd.read_csv("DATA/MSOA-Economic activity status.csv")
df_econ.head()

Index(['Middle layer Super Output Areas Code',
       'Middle layer Super Output Areas',
       'Economic activity status (20 categories) Code',
       'Economic activity status (20 categories)', 'Observation'],
      dtype='object')


,Middle layer Super Output Areas Code,Middle layer Super Output Areas,Economic activity status (20 categories) Code,Economic activity status (20 categories),Observation
0,E02000001,City of London 001,-8,Does not apply,579
1,E02000001,City of London 001,1,Economically active (excluding full-time stude...,440
2,E02000001,City of London 001,2,Economically active (excluding full-time stude...,3765
3,E02000001,City of London 001,3,Economically active (excluding full-time stude...,28
4,E02000001,City of London 001,4,Economically active (excluding full-time stude...,150


In [2]:
df_econ = df_econ.rename(columns={
    'Middle layer Super Output Areas Code': 'MSOA',
    'Economic activity status (20 categories)': 'status',
    'Observation': 'count'
})

In [ ]:
# change to numeric
df_econ['count'] = pd.to_numeric(df_econ['count'], errors='coerce')


In [ ]:
# divided unemployed or not
df_econ['is_unemployed'] = df_econ['status'].str.contains("Unemployed", case=False, na=False)


In [ ]:
# divided econ active or not
df_econ['is_econ_active'] = (
    df_econ['status'].str.contains("Economically active", case=False, na=False) &
    df_econ['status'].str.contains("In employment", case=False, na=False)
)
df_econ['is_inactive'] = df_econ['status'].str.startswith("Economically inactive", na=False)

In [ ]:
# the pop of unemploy
unemployed = df_econ[df_econ['is_unemployed']].groupby('MSOA')['count'].sum()

# the pop of eco active
econ_active = df_econ[df_econ['is_econ_active'] | df_econ['is_unemployed']].groupby('MSOA')['count'].sum()

# the pop of inactive
inactive = df_econ[df_econ['is_inactive']].groupby('MSOA')['count'].sum()


In [ ]:
# count of total pop
total = df_econ.groupby('MSOA')['count'].sum()

In [ ]:
# calculate the rate of umemployment and inactive
df_result = pd.DataFrame({
    'unemployment_rate': (unemployed / econ_active) * 100,
    'inactive_rate': (inactive / total) * 100
}).reset_index()

In [ ]:
# output
df_result.to_csv("cleaned/unemployment_inactive_cleaned.csv", index=False)
